In [1]:
!python -V

Python 3.9.13


In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [3]:
def read_dataframe(filename):
    
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
df_train = read_dataframe('../resources/inputs/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../resources/inputs/green_tripdata_2021-02.parquet')

In [5]:
len(df_train), len(df_val)

(73908, 61921)

In [6]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

7.75871520559622

In [10]:
with open('../resources/models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [11]:
lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

11.167275941179728

### MLFlow

In [12]:
import mlflow

mlflow.set_tracking_uri("sqlite:///../../mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [14]:
with mlflow.start_run():
    
    mlflow.set_tag("developer", "sebasjp")
    
    mlflow.log_param("train-data-path", "../resources/inputs/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "../resources/inputs/green_tripdata_2021-02.parquet")
    
    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    mlflow.log_metric("rmse", rmse)
    mlflow.log_artifact(local_path="../resources/models/lin_reg.bin",
                        artifact_path="models_pickle")

### Optimización bayesiana con hyperopt y XGBoost

In [15]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [16]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):
    with mlflow.start_run():
        
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=10,
            evals=[(valid, 'validation')],
            early_stopping_rounds=5
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [20]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

In [21]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=20,
    trials=Trials()
)

[07:28:40] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:14.75964                                                    
[1]	validation-rmse:10.98867                                                    
[2]	validation-rmse:8.89350                                                     
[3]	validation-rmse:7.78296                                                     
[4]	validation-rmse:7.20212                                                     
[5]	validation-rmse:6.89882                                                     
[6]	validation-rmse:6.73332                                                     
[7]	validation-rmse:6.64321                                                     
[8]	validation-rmse:6.58279                                                     
[9]	validation-rmse:6.54523                                                     
[07:28:43] WARNING: ../src/objective/regression_obj.cu:203: reg:linear 

[4]	validation-rmse:8.75739                                                     
[5]	validation-rmse:8.07340                                                     
[6]	validation-rmse:7.62487                                                     
[7]	validation-rmse:7.33050                                                     
[8]	validation-rmse:7.13565                                                     
[9]	validation-rmse:7.00993                                                     
[07:29:25] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:10.07758                                                    
[1]	validation-rmse:7.44344                                                     
[2]	validation-rmse:6.89377                                                     
[3]	validation-rmse:6.75877                                                     
[4]	validation-rmse:6.71779                                            

In [22]:
best_result

{'learning_rate': 0.8348727631802135,
 'max_depth': 79.0,
 'min_child_weight': 1.28514357799322,
 'reg_alpha': 0.017669633129935643,
 'reg_lambda': 0.07826001639366655}

### Entrenando con el mejor modelo y logueando con auto_log

In [15]:
params_ = {'learning_rate': 0.8348727631802135,
           'max_depth': 79,
           'min_child_weight': 1.28514357799322,
           'reg_alpha': 0.017669633129935643,
           'reg_lambda': 0.07826001639366655,
           'objective': 'reg:linear',
           'seed': 42}

mlflow.xgboost.autolog()
#mlflow.xgboost.autolog(disable=True)

booster = xgb.train(
            params=params_,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

2022/08/03 21:44:35 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '8381f0bdba5e4b32881f7d8c8675ab83', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[21:44:35] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:7.82433
[1]	validation-rmse:6.67313
[2]	validation-rmse:6.54469
[3]	validation-rmse:6.52136
[4]	validation-rmse:6.50962
[5]	validation-rmse:6.49796
[6]	validation-rmse:6.49041
[7]	validation-rmse:6.48323
[8]	validation-rmse:6.47603
[9]	validation-rmse:6.47077
[10]	validation-rmse:6.46502
[11]	validation-rmse:6.46377
[12]	validation-rmse:6.46018
[13]	validation-rmse:6.45509
[14]	validation-rmse:6.45059
[15]	validation-rmse:6.44681
[16]	validation-rmse:6.44269
[17]	validation-rmse:6.43714
[18]	validation-rmse:6.43789
[19]	validation-rmse:6.43421
[20]	validation-rmse:6.43067
[21]	validation-rmse:6.43187
[22]	validation-rmse:6.42977
[23]	validation-rmse:6.42935
[24]	validation-rmse:6.42936
[25]	validation-rmse:6.43103
[26]	validation-rmse:6.42929
[27]	validation-rmse:6.43386
[28]	validation-rmse:6.43169
[29]	validation-rmse:6.43064
[30]	validation-rmse:

2022/08/03 21:45:18 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.


---

In [23]:
mlflow.end_run()

### Log model 

In [18]:
with mlflow.start_run():
    
    params = {'learning_rate': 0.8348727631802135,
           'max_depth': 79,
           'min_child_weight': 1.28514357799322,
           'reg_alpha': 0.017669633129935643,
           'reg_lambda': 0.07826001639366655,
           'objective': 'reg:linear',
           'seed': 42}
    
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=10,
        evals=[(valid, 'validation')],
        early_stopping_rounds=5
    )
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)

    mlflow.set_tag("model", "xgboost")
    mlflow.log_params(params)
    mlflow.log_metric("rmse", rmse)
    
    with open('../resources/models/preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)
    
    mlflow.log_artifact(local_path="../resources/models/preprocessor.b",
                        artifact_path="preprocessor")
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[11:42:51] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:7.82433
[1]	validation-rmse:6.67313
[2]	validation-rmse:6.54469
[3]	validation-rmse:6.52136
[4]	validation-rmse:6.50962
[5]	validation-rmse:6.49796
[6]	validation-rmse:6.49041
[7]	validation-rmse:6.48323
[8]	validation-rmse:6.47603
[9]	validation-rmse:6.47077


/home/sebastian/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
